# Gradient Boosting (classifier and regression)

### Decision trees
* 2 decisions: what feature to split and where to split
* Root node = entire dataset; decision node = when the data is split; leaf = where the data is categorized/regressed
* Typically requires descretized data so that you can compare score of a split location without having to calculate the score for an inordinate number of possible split locations.
* Typical splitting criteria are entropy decrease, information gain, or similarity gain among the groups
  * Similarity score can be calculated as $(sum\_of\_ residuals)^2/(num\_residuals + \lambda)$ where residuals are from the mean of the root node, which is the prediction. When there are values on both sides of this prediction, they will cancel and make this similarity score lower (Starmer).
  * The similarity gain is: (similarity of the left node) + (similarity of right node) - (similarity of root) (Starmer).
  * The similarity gain is calculated for all possible splits and then the highest gain is selected
* Stopping criteria is typically that a less than a minimum number of data at a node converts it to a leaf
* Pruning is done to stop overfitting by removing nodes that do not contribute to the reduction of the cost function (ie accuracy) (Seif)
* Cost function is typically $\sum (Y-\hat{Y})^2$ for regression or Gini index function $\sum (p_{k}*(1-p_{k}))$ where $p_{k}$ is the proportion of a leaf node's samples of each class (Seif)
* The leaf nodes are assigned based on the class what most of the samples in the node

### Bagging
* Sample m times with replacement from the original train dataset (of size $n$), with each of the $m$ samples having $n'$ size and train 1 decision tree on each sample
  * Typically $n=n'$. According to Aslam, Popa, and Rivest there is a $1-1/e$ (63%) of the original training dataset being represented in each sample in this case.
  * Replacement occurs after each one that you pick. E.g., if you have 3 marbles in your dataset (1 red, 1 blue, 1 green) and are choosing two, if you pick red once, you may pick it again to make a sample of 2 red marbles.
* Bagging works because it equalzes the "leverage" of each example to stop outliers or particular examples from having too great an impact. It does this because those high leverage examples are few in number, so they are often left out of bagging samples and therefore many models are produced without the effect of these high leverage examples, therefore the impact of the sample is reduced.
* Each individual tree is combined so that it can "vote" on the ultimate prediction

### Boosting
* Construct decision trees to ensemble in a forest like in boosting
* However, the output of the previous tree is used in the next. Specifically, the ones that were wrong from the previous tree are weighted higher as the input the next one
* Gradient boosting is the method that measures which samples contributed more to the cost function by taking the derivative of the loss funciton with respect to the sample. This is similar to how it was used in neural networks.

<center>Resources</center>

http://cs229.stanford.edu/notes/cs229-notes-ensemble.pdf

http://people.csail.mit.edu/rivest/pubs/APR07.pdf

http://www.math.univ-toulouse.fr/~agarivie/Telecom/apprentissage/articles/BaggingML.pdf

https://towardsdatascience.com/a-guide-to-decision-trees-for-machine-learning-and-data-science-fe2607241956

https://www.youtube.com/watch?v=OtD8wVaFm6E

Chen, T., & Guestrin, C. (2016). XGBoost: A Scalable Tree Boosting System. In Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (pp. 785–794). ACM.

In [34]:
# basic decision trees

# for all values of the split calculate cross-entropy to the version where there is no split
# If the maximum cross entropy is above a threshhold, split and split at that value
# To find the most optimal value, take the mean if the two biggest cross entropy values are sequential
# The two distributions being compared are the true distribution for a sample vs the one predicted by the decision tree
# One simple application is do do the cross entropy class by class

from math import log2
import numpy as np

def cross_entropy(p, q):
    # p & q are distributions representing the same thing
    # Cross entropy measures the average total number of bits from Q instead of P
    # This can be broken down into the number of bits to represent P and the difference in bits from P to Q
        # Therefore can say: E(P, Q) = E(P) + KL(P||Q) where KL is the extra bits 
    assert p.ndim == 1
    assert p.ndim == q.ndim
    # Elementwise sum of p_i * log2(q_i) for all i
    return -1*np.sum(p*log2(q))

def simple_dtrees(X, y, termination_tolerance, num_alternate_tresholds, alternate_threshold_step):
    # The decision criteria are stored in a matrix, where each row is the depth and each column is the 
    # Each decision criteria contains 3 items, previous decision status (0 or 1), 
    criteria = [[[]]]
    cat_dict = {}
    while total_ce < termination_tolerance:
        for x in X:
            for layer in criteria:
                pass
                
            
    

In [7]:
rng = np.random.default_rng()
X = np.array(samples[0], copy = True)
print(X.shape)
chosen = rng.choice(X, size = 50, replace = False)

(100, 20)


In [27]:
import pandas as pd
import pathlib
from sklearn.model_selection import train_test_split

ads_df = pd.read_csv(pathlib.Path(r'./datasets/Introduction to Statistical Learning with Applications in R/Advertising.csv'))
ads_df.head()
ads_df = ads_df[ads_df.columns[1:]]
ads_df.head()
data = ads_df.values[:, :-1]; label = ads_df.values[:,-1]
data_train, data_test, label_train, label_test = train_test_split(data, label ,test_size = .6)

In [35]:
import xgboost as xgb
from math import sqrt

dtrain = xgb.DMatrix(data_train, label=label_train)

# All parameters: https://xgboost.readthedocs.io/en/latest/parameter.html
# eta: step size
# gamma: how much loss reduction to get division
# max_depth: how many layers of nodes past the root the tree can have
param = {'max_depth':2, 'eta':1,}
num_round = 2
bst = xgb.train(param, dtrain, num_round)
dtest = xgb.DMatrix(data_test)
labelpred = bst.predict(dtest)

print('RMSE: ', sqrt(sum((label_test - labelpred)**2)/len(label_test)))
print('avg sales for comparison: ', np.mean(label))

RMSE:  2.4812375084350458
avg sales for comparison:  14.0225


### 1. Type of Data
* Categorical or continuous
  * Note: the output space for regression decision trees is not continuous like with other methods, such as sigmoid regression

### 2. Use Case
* When it is acceptable not to have a continuous output space

### 3. Application
* Almost any regression or classification task satisfying the above
* Especially when training time or possible overfitting is not relevant

### 4. Basic Concept
* Gradient boosting uses multiple decision trees trained sequentially, where the examples that were poorly categorized in the previous tree are weighted higher in the next tree.

### 5. Assumptions
* Worthwhile splits exist

### 6. Existing solutions
* XGBoost, LighGBM (which is a method that is more efficient in its sample dropout stage by not dropping out samples with high gradient (that we have a lot to learn from)), catboost

### 7. Strengths and Weaknesses
#### Strengths
* Highly accurate
* Not as data or resource hungry as deep learning algorithms

#### Weaknesses
* Prone to overfit
* Non-continuous output